<a href="https://colab.research.google.com/github/joni901/chatboot/blob/main/Chaatbot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF transformers spacy nltk simpletransformers language-tool-python rake_nltk  sentence-transformers torch
!pip install -U sentence-transformers pinecone-client
!python -m spacy download pt_core_news_sm
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 50.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import fitz  # PyMuPDF
import re
import torch
import spacy
import unicodedata
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import language_tool_python
from rake_nltk import Rake
import nltk
import spacy
import matplotlib.pyplot as plt
import numpy as np
import pinecone
from sklearn.preprocessing import normalize
import tensorflow as tf
import tensorflow_hub as hub

# Inicializar Spacy e NLTK

# Inicializar LanguageTool
tool = language_tool_python.LanguageTool('pt-BR')

# Função para corrigir texto usando LanguageTool
def corrigir_texto(texto):
    matches = tool.check(texto)
    texto_corrigido = language_tool_python.utils.correct(texto, matches)
    return texto_corrigido

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

pdf_path = "/content/Relatório Final de Projetos Propostos 2021 com errata (1).pdf"
text = extract_text_from_pdf(pdf_path)



# Função para limpar o texto
def limpar_texto(texto):
    texto_limpo = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    texto_limpo = re.sub(r'\d+/\d+\s*-\s*CADERNOS DE ELEI.*?2013', '', texto_limpo)
    texto_limpo = re.sub(r'\s+', ' ', texto_limpo)
    texto_limpo = re.sub(r'\n', ' ', texto_limpo)

    return texto_limpo.strip()

# Carregar o modelo de transformadores de sentenças

def chunk_text(text, model):
    # Dividir o texto em sentenças individuais
    single_sentences = re.split(r'(?<=\.|\?)\s', text)
    ss = [limpar_texto(i) for i in single_sentences]

    # Preparar os dados para processamento
    sentences = [{"sentence": x, "index": y} for y, x in enumerate(ss)]

    # Função para combinar sentenças adjacentes
    def combine_sentences(sentences, max_chunk_size=1):
        combined_sentences = []
        for i in range(len(sentences)):
            combined_sentence = sentences[i]["sentence"]
            for j in range(1, max_chunk_size + 1):
                if i + j < len(sentences):
                    combined_sentence += " " + sentences[i + j]["sentence"]
            combined_sentences.append(combined_sentence)
        return combined_sentences

    combined_sentences = combine_sentences(sentences)

    # Obter embeddings para as sentenças combinadas
    embeddings = model(combined_sentences).numpy()

    for i, x in enumerate(sentences):
        sentences[i]["combined_sentences_embedding"] = embeddings[i]

    # Função para calcular distâncias cosseno entre embeddings
    def calculate_cosine_distances(sentences):
        distances = []
        for i in range(len(sentences) - 1):
            embedding_current = sentences[i]["combined_sentences_embedding"]
            embedding_next = sentences[i + 1]["combined_sentences_embedding"]

            similarity = cosine_similarity([embedding_current], [embedding_next])[0][0]
            distance = 1 - similarity
            distances.append(distance)
            sentences[i]["distance_next"] = distance
        return distances, sentences

    distances, sentences = calculate_cosine_distances(sentences)

    breakpoint_percentile_threshold = 80

    breakpoint_percentile = np.percentile(distances, breakpoint_percentile_threshold)

    # Identificar os índices dos pontos de quebra
    indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_percentile]

    # Marcar os chunks no gráfico
    start_index = 0
    chunks = []
    for index in indices_above_thresh:
        end_index = index
        group = sentences[start_index:end_index + 1]
        combined_text = " ".join([x["sentence"] for x in group])
        chunks.append(combined_text.strip())
        start_index = end_index + 1

    if start_index < len(sentences):
        group = sentences[start_index:]
        combined_text = " ".join([x["sentence"] for x in group])
        chunks.append(combined_text.strip())

    return chunks

model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


chunks = chunk_text(text, model)
for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i}: {chunk}\n")


# Função para extrair palavras-chave usando Rake
def extrair_palavras_chave_rake(texto, top_n=5):
    rake = Rake(language='portuguese')
    rake.extract_keywords_from_text(texto)
    palavras_chave = rake.get_ranked_phrases()[:top_n]
    return palavras_chave

# Carregar modelo T5 e tokenizer para geração de títulos
model_name = "pierreguillou/t5-base-qa-squad-v1.1-portuguese"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Função para gerar título a partir do texto
def generate_title(text, max_length=20):
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=max_length, num_beams=4, early_stopping=True)
    title = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return title


# Dividindo o texto em partes gerenciáveis



generated_titles = {idx: generate_title(content) for idx, content in enumerate(chunks)}


document_titles = list(generated_titles.values())






Chunk 1: Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br RELATORIO DA ANALISE DE PROJETOS PROPOSTOS NO ANO DE 20211 INTRODUCAO O grupo de monitoramento do legislativo do Observatorio Social do Brasil de Sao Paulo realizou o levantamento e a analise da atividade de projetos propostos do ano de 2021, de forma igual ao que vinha realizando desde o ano de 2017. Como metodologia para a realizacao da atividade em 2021, foram distribuidas entre os voluntarios planilhas individuais dos vereadores, para o levantamento dos projetos propostos (Fonte: https://splegisconsulta.saopaulo.sp.leg.br/) e sua classificacao em categorias, pre-estabelecidas pela equipe. Tendo em vista que as categorias trazem um certo grau de subjetividade, apos os dados serem levantados, as planilhas foram enviadas para os vereadores de Sao Paulo (Incluindo os suplentes que chegaram a ser empossados em

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preparar o vetorizador TF-IDF
vectorizer = TfidfVectorizer()


def select_best_chunk(chunks, question, vectorizer):
    """Seleciona o melhor trecho com base na similaridade de cosseno com a pergunta."""
    if not chunks:
        return ""
    vectorizer.fit(chunks)
    question_vector = vectorizer.transform([question.lower()])
    chunk_vectors = vectorizer.transform([chunk.lower() for chunk in chunks])
    similarities = cosine_similarity(question_vector, chunk_vectors).flatten()
    best_index = similarities.argmax()
    return chunks[best_index]




In [ ]:
trains_data = [
   {
        "context": "Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br RELATORIO DA ANALISE DE PROJETOS PROPOSTOS NO ANO DE 20211 INTRODUCAO O grupo de monitoramento do legislativo do Observatorio Social do Brasil de Sao Paulo realizou o levantamento e a analise da atividade de projetos propostos do ano de 2021, de forma igual ao que vinha realizando desde o ano de 2017. Como metodologia para a realizacao da atividade em 2021, foram distribuidas entre os voluntarios planilhas individuais dos vereadores, para o levantamento dos projetos propostos (Fonte: https://splegisconsulta.saopaulo.sp.leg.br/) e sua classificacao em categorias, pre-estabelecidas pela equipe. Tendo em vista que as categorias trazem um certo grau de subjetividade, apos os dados serem levantados, as planilhas foram enviadas para os vereadores de Sao Paulo (Incluindo os suplentes que chegaram a ser empossados em algum momento do ano e assinaram projetos legislativos) para confirmacao das informacoes. Entretanto o retorno a estas solicitacoes foi baixissimo, com somente cinco legisladores tendo dado respostas dentro do prazo estipulado, denotando falta de interesse dos parlamentares em atender a pedidos de informacoes da sociedade civil organizada. A analise a seguir, portanto, levou em consideracao todas as demais informacoes levantadas como devidamente confirmadas, com as planilhas individuais dos vereadores, a planilha geral estando presente em 1 Errata: O total de assinaturas de projetos propostos e de 2164 e nao de 2176 como apontava o relatorio anteriormente. Tambem a area com mais assinaturas e a de desenvolvimento social e nao frente parlamentar, que na verdade e a segunda mais assinada. As correcoes foram possiveis devido a montagem do painel de projetos propostos, que permitiu maior precisao na contagem dos dados quantitativos na atividade e sera publicado em breve.",
        "qas": [
            {
                "id": "1",
                "question": "Qual foi o objetivo principal do relatório de análise dos projetos propostos em 2021?",
                "answers": [
                    {
                        "text": "Avaliar a atividade legislativa do ano de 2021, identificando tendências, categorias mais propostas e a atuação dos vereadores em termos de assinaturas em projetos.",
                        "answer_start": 223
                    }
                ],
                "is_impossible": False
            }
        ]
    },
   {
    "context": "Educacao Basica PR 2/2021 LGBTQIA+ por Direitos e Cidadania. Apoio e Desenvolvimento aos setores Turistico, Hoteleiro, Gastronomico, de Sistemas Expositores, de Eventos, Negocios e de Lazer, e da outras providencias Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Tambem no ano de 2021, em numero absolutos, o total de assinaturas em areas como protecao dos animais e seguranca publica chegam a quase o total feito nas proposicoes que ocorreram na legislatura anterior em ambas. Ja em numeros percentuais, quando comparados com 2020, proporcionalmente as assinaturas na area de protecao dos animais subiram de 1,12% em 2020 para 3,10% do total em 2021; ja em seguranca publica o numero salta de 0,84% em 2020 para 2,68% em 2021. Outras areas que tiveram aumento percentual no total de assinaturas entre 2020 e 2021 foram meio ambiente, habitacao e urbanismo e educacao e cultura que subiram, respectivamente, de 1,33% para 3,00%, de 4,27% para 6,51% e de 8,34% para 9,06%, alem de todas estas categorias terem aumentado em numeros absolutos. Quando se trata de projetos de baixo impacto, as categorias que comumente mais se destacam sao as relacionadas a denominacao de logradouro e a datas comemorativas e homenagens diversas. Na legislatura anterior ja houve anos nos quais estas duas categorias chegaram a receber quase metade do total de assinaturas, sendo 46% do total em 2018 e 43% em 2019. E esta queda no percentual das mesmas no total de assinaturas continuou em 2020, indo para 23%, e posteriormente em 2021 passaram a ser 20,84%. Quando se trata de numeros absolutos, nos anos de 2017, 2018 e 2019, a area de datas comemorativas e homenagens diversas, em 2019 foi a mais assinada. Ja em 2020, quando desenvolvimento social foi a categoria mais assinada, ela caiu para a quarta colocacao e em 2021 foi para quinto lugar, quando a area de frente parlamentar foi a mais assinada. Por fim, outro ponto a destacar sao as assinaturas em projetos nas categorias de combate a corrupcao e transparencia. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br No tocante aos projetos de lei na area de combate a corrupcao a quantidade de assinaturas permaneceu baixa em 2021, com um total de 8 assinaturas, e os projetos nesta area giraram em torno da obrigatoriedade da prestacao de informacoes ao legislativo sobre alteracoes das tarifas do servico transporte publico de passageiros do municipio, da retomada da gestao administrativa e pedagogica de equipamentos repassados a gestao indireta por associacoes e OSCIPs para o provimento da educacao infantil de 0 a 3 anos de idade e da politica municipal de fiscalizacao, prevencao e combate a adulteracao e comercializacao de combustiveis adulterados. Quando comparada com 2017, ano em que esse valor de assinaturas era de 26, considera-se um numero baixo. E sobre os baixos numeros de assinaturas encontrados na area de combate a corrupcao, algumas suposicoes podem ser feitas baseadas tanto no contexto de epoca, assim como no desenrolar dos anos seguintes. Vale destacar que em meados de 2017 o Brasil ainda vivia as atividades da Operacao Lava Jato, que se apresentou como uma operacao que visava combater a corrupcao, sendo que mesma tambem foi uma das principais motivadoras do surgimento de individuos, grupos politicos e movimentos que defendiam o enfrentamento a praticas corruptas na sociedade. No entanto, o que se observou nos anos seguintes foi o progressivo esvaziamento desta causa, o que leva a supor que a bandeira do combate a corrupcao nao passou de mera retorica que serviu para promover os projetos de poder de alguns setores da sociedade. E isto nao foi diferente na Camara dos Vereadores de Sao Paulo, porque no ano de 2018 nao houve nenhuma assinatura em projetos para esta area, em 2019 este numero sobe para 1, em 2020 aumenta para 9 e em 2021 cai para 8. O dever de agora e observar se o numero de assinaturas em projetos de combate a corrupcao vai aumentar ou se vai estagnar ou diminuir. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Ja o numero de assinaturas em projetos de lei na area de transparencia no ano de 2021 foi de 23 ao todo. Quando comparado com a sequencia historica, que surgiu em 2017, este chega a ser um numero alto para a area, ficando atras apenas do ano de 2019 quando houve 35 assinaturas ao todo. Na area de transparencia os projetos giraram em torno da consolidacao da politica de dados abertos e transparencia no ambito do municipio, combate a noticias falsas, digitalizacao de documentos fisicos do municipio, a criacao de um observatorio de dados abertos do municipio e divulgacao de informacoes relacionadas a obras, contaminacao por covid-19, vacinacao, contratos de terceirizacao de servicos e os processos administrativos relacionados a regularizacao fundiaria. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br CONSIDERACOES FINAIS E valido e importante destacar que o exercicio do controle social e extremamente necessario para saber se as instituicoes estao em bom funcionamento e se mantem sintonia com as demandas e a realidade da sociedade, alem de ajudar a sociedade a propor caminhos para melhorar a gestao publica, a fomentar a participacao social na administracao publica e a levar conhecimento ao publico.",
    "qas": [
        {
            "question": "Quais foram as duas áreas com maior número de assinaturas em 2021?",
            "id": "2",

            "answers": [
                {
                    "text": "Desenvolvimento social (12,66%) e saúde e esporte (11,41%).",
                    "answer_start": 356
                }
            ],
             'is_impossible': False,
        }
    ]
   },
{
    "context": "Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br RELATORIO DA ANALISE DE PROJETOS PROPOSTOS NO ANO DE 20211 INTRODUCAO O grupo de monitoramento do legislativo do Observatorio Social do Brasil de Sao Paulo realizou o levantamento e a analise da atividade de projetos propostos do ano de 2021, de forma igual ao que vinha realizando desde o ano de 2017. Como metodologia para a realizacao da atividade em 2021, foram distribuidas entre os voluntarios planilhas individuais dos vereadores, para o levantamento dos projetos propostos (Fonte: https://splegisconsulta.saopaulo.sp.leg.br/) e sua classificacao em categorias, pre-estabelecidas pela equipe. Tendo em vista que as categorias trazem um certo grau de subjetividade, apos os dados serem levantados, as planilhas foram enviadas para os vereadores de Sao Paulo (Incluindo os suplentes que chegaram a ser empossados em algum momento do ano e assinaram projetos legislativos) para confirmacao das informacoes. Entretanto o retorno a estas solicitacoes foi baixissimo, com somente cinco legisladores tendo dado respostas dentro do prazo estipulado, denotando falta de interesse dos parlamentares em atender a pedidos de informacoes da sociedade civil organizada. A analise a seguir, portanto, levou em consideracao todas as demais informacoes levantadas como devidamente confirmadas, com as planilhas individuais dos vereadores, a planilha geral estando presente em 1 Errata: O total de assinaturas de projetos propostos e de 2164 e nao de 2176 como apontava o relatorio anteriormente. Tambem a area com mais assinaturas e a de desenvolvimento social e nao frente parlamentar, que na verdade e a segunda mais assinada. As correcoes foram possiveis devido a montagem do painel de projetos propostos, que permitiu maior precisao na contagem dos dados quantitativos na atividade e sera publicado em breve.",
    "qas": [
        {
            "question": "Como os voluntários do Observatório Social do Brasil realizaram a coleta de dados para o relatório?",
            "id": "3",
            "answers": [
                {
                    "text": "Por meio de planilhas individuais distribuídas entre os voluntários, que analisaram as informações do site da Câmara Municipal de São Paulo.",
                    "answer_start": 374
                }
            ],
            "is_impossible": False
        }
    ]
},
{
    "context": "PR 3/2021 Educacao Basica PR 2/2021 LGBTQIA+ por Direitos e Cidadania. Apoio e Desenvolvimento aos setores Turistico, Hoteleiro, Gastronomico, de Sistemas Expositores, de Eventos, Negocios e de Lazer, e da outras providencias Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Tambem no ano de 2021, em numero absolutos, o total de assinaturas em areas como protecao dos animais e seguranca publica chegam a quase o total feito nas proposicoes que ocorreram na legislatura anterior em ambas. Ja em numeros percentuais, quando comparados com 2020, proporcionalmente as assinaturas na area de protecao dos animais subiram de 1,12% em 2020 para 3,10% do total em 2021; ja em seguranca publica o numero salta de 0,84% em 2020 para 2,68% em 2021. Outras areas que tiveram aumento percentual no total de assinaturas entre 2020 e 2021 foram meio ambiente, habitacao e urbanismo e educacao e cultura que subiram, respectivamente, de 1,33% para 3,00%, de 4,27% para 6,51% e de 8,34% para 9,06%, alem de todas estas categorias terem aumentado em numeros absolutos. Quando se trata de projetos de baixo impacto, as categorias que comumente mais se destacam sao as relacionadas a denominacao de logradouro e a datas comemorativas e homenagens diversas. Na legislatura anterior ja houve anos nos quais estas duas categorias chegaram a receber quase metade do total de assinaturas, sendo 46% do total em 2018 e 43% em 2019. E esta queda no percentual das mesmas no total de assinaturas continuou em 2020, indo para 23%, e posteriormente em 2021 passaram a ser 20,84%. Quando se trata de numeros absolutos, nos anos de 2017, 2018 e 2019, a area de datas comemorativas e homenagens diversas, em 2019 foi a mais assinada. Ja em 2020, quando desenvolvimento social foi a categoria mais assinada, ela caiu para a quarta colocacao e em 2021 foi para quinto lugar, quando a area de frente parlamentar foi a mais assinada. Por fim, outro ponto a destacar sao as assinaturas em projetos nas categorias de combate a corrupcao e transparencia. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br No tocante aos projetos de lei na area de combate a corrupcao a quantidade de assinaturas permaneceu baixa em 2021, com um total de 8 assinaturas, e os projetos nesta area giraram em torno da obrigatoriedade da prestacao de informacoes ao legislativo sobre alteracoes das tarifas do servico transporte publico de passageiros do municipio, da retomada da gestao administrativa e pedagogica de equipamentos repassados a gestao indireta por associacoes e OSCIPs para o provimento da educacao infantil de 0 a 3 anos de idade e da politica municipal de fiscalizacao, prevencao e combate a adulteracao e comercializacao de combustiveis adulterados. Quando comparada com 2017, ano em que esse valor de assinaturas era de 26, considera-se um numero baixo. E sobre os baixos numeros de assinaturas encontrados na area de combate a corrupcao, algumas suposicoes podem ser feitas baseadas tanto no contexto de epoca, assim como no desenrolar dos anos seguintes. Vale destacar que em meados de 2017 o Brasil ainda vivia as atividades da Operacao Lava Jato, que se apresentou como uma operacao que visava combater a corrupcao, sendo que mesma tambem foi uma das principais motivadoras do surgimento de individuos, grupos politicos e movimentos que defendiam o enfrentamento a praticas corruptas na sociedade. No entanto, o que se observou nos anos seguintes foi o progressivo esvaziamento desta causa, o que leva a supor que a bandeira do combate a corrupcao nao passou de mera retorica que serviu para promover os projetos de poder de alguns setores da sociedade. E isto nao foi diferente na Camara dos Vereadores de Sao Paulo, porque no ano de 2018 nao houve nenhuma assinatura em projetos para esta area, em 2019 este numero sobe para 1, em 2020 aumenta para 9 e em 2021 cai para 8. O dever de agora e observar se o numero de assinaturas em projetos de combate a corrupcao vai aumentar ou se vai estagnar ou diminuir. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Ja o numero de assinaturas em projetos de lei na area de transparencia no ano de 2021 foi de 23 ao todo. Quando comparado com a sequencia historica, que surgiu em 2017, este chega a ser um numero alto para a area, ficando atras apenas do ano de 2019 quando houve 35 assinaturas ao todo. Na area de transparencia os projetos giraram em torno da consolidacao da politica de dados abertos e transparencia no ambito do municipio, combate a noticias falsas, digitalizacao de documentos fisicos do municipio, a criacao de um observatorio de dados abertos do municipio e divulgacao de informacoes relacionadas a obras, contaminacao por covid-19, vacinacao, contratos de terceirizacao de servicos e os processos administrativos relacionados a regularizacao fundiaria. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br CONSIDERACOES FINAIS E valido e importante destacar que o exercicio do controle social e extremamente necessario para saber se as instituicoes estao em bom funcionamento e se mantem sintonia com as demandas e a realidade da sociedade, alem de ajudar a sociedade a propor caminhos para melhorar a gestao publica, a fomentar a participacao social na administracao publica e a levar conhecimento a todos.",
    "qas": [
        {
            "question": "Quais foram as áreas mais assinadas em projetos no ano de 2021?",
            "id": "4",
            "answers": [
                {
                    "text": "A área mais assinada em 2021 foi a de frente parlamentar, seguida pela área de desenvolvimento social, proteção dos animais e segurança pública.",
                    "answer_start": 846
                }
            ],
            "is_impossible": False
        }
    ]
},



  {
    "context": "Q PR 3/2021 Educacao Basica PR 2/2021 LGBTQIA+ por Direitos e Cidadania. Apoio e Desenvolvimento aos setores Turistico, Hoteleiro, Gastronomico, de Sistemas Expositores, de Eventos, Negocios e de Lazer, e da outras providencias Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Tambem no ano de 2021, em numero absolutos, o total de assinaturas em areas como protecao dos animais e seguranca publica chegam a quase o total feito nas proposicoes que ocorreram na legislatura anterior em ambas. Ja em numeros percentuais, quando comparados com 2020, proporcionalmente as assinaturas na area de protecao dos animais subiram de 1,12% em 2020 para 3,10% do total em 2021; ja em seguranca publica o numero salta de 0,84% em 2020 para 2,68% em 2021. Outras areas que tiveram aumento percentual no total de assinaturas entre 2020 e 2021 foram meio ambiente, habitacao e urbanismo e educacao e cultura que subiram, respectivamente, de 1,33% para 3,00%, de 4,27% para 6,51% e de 8,34% para 9,06%, alem de todas estas categorias terem aumentado em numeros absolutos. Quando se trata de projetos de baixo impacto, as categorias que comumente mais se destacam sao as relacionadas a denominacao de logradouro e a datas comemorativas e homenagens diversas. Na legislatura anterior ja houve anos nos quais estas duas categorias chegaram a receber quase metade do total de assinaturas, sendo 46% do total em 2018 e 43% em 2019. E esta queda no percentual das mesmas no total de assinaturas continuou em 2020, indo para 23%, e posteriormente em 2021 passaram a ser 20,84%. Quando se trata de numeros absolutos, nos anos de 2017, 2018 e 2019, a area de datas comemorativas e homenagens diversas, em 2019 foi a mais assinada. Ja em 2020, quando desenvolvimento social foi a categoria mais assinada, ela caiu para a quarta colocacao e em 2021 foi para quinto lugar, quando a area de frente parlamentar foi a mais assinada. Por fim, outro ponto a destacar sao as assinaturas em projetos nas categorias de combate a corrupcao e transparencia. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br No tocante aos projetos de lei na area de combate a corrupcao a quantidade de assinaturas permaneceu baixa em 2021, com um total de 8 assinaturas, e os projetos nesta area giraram em torno da obrigatoriedade da prestacao de informacoes ao legislativo sobre alteracoes das tarifas do servico transporte publico de passageiros do municipio, da retomada da gestao administrativa e pedagogica de equipamentos repassados a gestao indireta por associacoes e OSCIPs para o provimento da educacao infantil de 0 a 3 anos de idade e da politica municipal de fiscalizacao, prevencao e combate a adulteracao e comercializacao de combustiveis adulterados. Quando comparada com 2017, ano em que esse valor de assinaturas era de 26, considera-se um numero baixo. E sobre os baixos numeros de assinaturas encontrados na area de combate a corrupcao, algumas suposicoes podem ser feitas baseadas tanto no contexto de epoca, assim como no desenrolar dos anos seguintes. Vale destacar que em meados de 2017 o Brasil ainda vivia as atividades da Operacao Lava Jato, que se apresentou como uma operacao que visava combater a corrupcao, sendo que mesma tambem foi uma das principais motivadoras do surgimento de individuos, grupos politicos e movimentos que defendiam o enfrentamento a praticas corruptas na sociedade. No entanto, o que se observou nos anos seguintes foi o progressivo esvaziamento desta causa, o que leva a supor que a bandeira do combate a corrupcao nao passou de mera retorica que serviu para promover os projetos de poder de alguns setores da sociedade. E isto nao foi diferente na Camara dos Vereadores de Sao Paulo, porque no ano de 2018 nao houve nenhuma assinatura em projetos para esta area, em 2019 este numero sobe para 1, em 2020 aumenta para 9 e em 2021 cai para 8. O dever de agora e observar se o numero de assinaturas em projetos de combate a corrupcao vai aumentar ou se vai estagnar ou diminuir. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Ja o numero de assinaturas em projetos de lei na area de transparencia no ano de 2021 foi de 23 ao todo. Quando comparado com a sequencia historica, que surgiu em 2017, este chega a ser um numero alto para a area, ficando atras apenas do ano de 2019 quando houve 35 assinaturas ao todo. Na area de transparencia os projetos giraram em torno da consolidacao da politica de dados abertos e transparencia no ambito do municipio, combate a noticias falsas, digitalizacao de documentos fisicos do municipio, a criacao de um observatorio de dados abertos do municipio e divulgacao de informacoes relacionadas a obras, contaminacao por covid-19, vacinacao, contratos de terceirizacao de servicos e os processos administrativos relacionados a regularizacao fundiaria. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br CONSIDERACOES FINAIS E valido e importante destacar que o exercicio do controle social e extremamente necessario para saber se as instituicoes estao em bom funcionamento e se mantem sintonia com as demandas e a realidade da sociedade, alem de ajudar a sociedade a propor caminhos para melhorar a gestao publica, a fomentar a participacao social na administracao publica e a levar conhecimento ao publico.",
    "qas": [
      {
        "question": "Qual foi o número total de assinaturas de projetos propostos em 2021?",
        "id": "5",
        "answers": [
          {
            "text": "2164 assinaturas.",
            "answer_start": 843
          }
        ],
        "is_impossible": False
      }
    ]
  },
  {
    "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%.",
    "qas": [
      {
        "question": "Quais categorias tiveram aumentos significativos nas assinaturas em 2021, quando comparadas a 2020?",
        "id": "6",
        "answers": [
          {
            "text": "Desenvolvimento social (de 10,14% para 12,66%), saúde e esporte (de 9,15% para 11,41%), proteção dos animais (de 1,12% para 3,10%), e segurança pública (de 0,84% para 2,68%).",
            "answer_start": 124
          }
        ],
        "is_impossible": False
      }
    ]
  },

{
    "context": "De acordo com a CMSP: Grupos suprapartidarios de atuacao vol- tada, em conjunto com a sociedade civil e outros orgaos publicos, ao debate e aprimoramento da legislacao e de politicas publicas no am- bito de sua competencia. Tem tratamento autonomo em relacao a qualquer Comissao Permanente ou Temporaria. Atuam dentro ou fora das dependencias da Camara Municipal, de acordo com seu proposi- to (fonte: https://www.saopaulo.sp.leg.br/atividade-legislativa/frentes- parlamentares/). As 271 assinaturas foram distribuidas em 34 projetos, conforme se verifica no quadro 3, a seguir, que tambem destaca os temas objeto da criacao das frentes parlamentares ao longo do ano de 2021. Note-se que algumas das frentes foram criadas com o mesmo objeto, e que nem todas foram efetivamente transformadas em normas, o que nos leva a algumas reflexoes: - Qual a intencao dos vereadores ao criar tais frentes? - Qual a tramitacao e participacao da sociedade civil? - Existe uma conclusao da Frente Parlamentar, com apresentacao de eventuais resultados obtidos? Outra constatacao interessante (a conferir posteriormente) e que, comparando-se o quadro 1 com o quadro 3, verificam-se muitos projetos de Frente Parlamentar no primeiro ano de mandato.",
    "qas": [
        {
            "question": "O que são Frentes Parlamentares, segundo a definição fornecida no relatório?",
            "id": "7",
            "answers": [
                {
                    "text": "Grupos suprapartidários que atuam com a sociedade civil e outros órgãos públicos para debater e aprimorar legislações e políticas públicas em suas competências.",
                    "answer_start": 35
                }
            ],
            "is_impossible": False
        }
    ]
},
{
    "context": "De acordo com a CMSP: Grupos suprapartidarios de atuacao vol- tada, em conjunto com a sociedade civil e outros orgaos publicos, ao debate e aprimoramento da legislacao e de politicas publicas no am- bito de sua competencia. Tem tratamento autonomo em relacao a qualquer Comissao Permanente ou Temporaria. Atuam dentro ou fora das dependencias da Camara Municipal, de acordo com seu proposi- to (fonte: https://www.saopaulo.sp.leg.br/atividade-legislativa/frentes- parlamentares/). As 271 assinaturas foram distribuidas em 34 projetos, conforme se verifica no quadro 3, a seguir, que tambem destaca os temas objeto da criacao das frentes parlamentares ao longo do ano de 2021. Note-se que algumas das frentes foram criadas com o mesmo objeto, e que nem todas foram efetivamente transformadas em normas, o que nos leva a algumas reflexoes: - Qual a intencao dos vereadores ao criar tais frentes? - Qual a tramitacao e participacao da sociedade civil? - Existe uma conclusao da Frente Parlamentar, com apresentacao de eventuais resultados obtidos? Outra constatacao interessante (a conferir posteriormente) e que, comparando-se o quadro 1 com o quadro 3, verificam-se muitos projetos de Frente Parlamentar no primeiro ano de mandato.",
    "qas": [
        {
            "question": "Quais reflexões o relatório traz sobre a criação de Frentes Parlamentares?",
            "id": "8",
            "answers": [
                {
                    "text": "Questiona a intenção dos vereadores, a participação da sociedade civil e os resultados apresentados pelas frentes criadas.",
                    "answer_start": 590
                }
            ],
            "is_impossible": False
        }
    ]
},


    {
        "context": "Quando se trata de projetos de baixo impacto, as categorias que comumente mais se destacam sao as relacionadas a denominacao de logradouro e a datas comemorativas e homenagens diversas. Na legislatura anterior ja houve anos nos quais estas duas categorias chegaram a receber quase metade do total de assinaturas, sendo 46% do total em 2018 e 43% em 2019. E esta queda no percentual das mesmas no total de assinaturas continuou em 2020, indo para 23%, e posteriormente em 2021 passaram a ser 20,84%. Quando se trata de numeros absolutos, nos anos de 2017, 2018 e 2019, a area de datas comemorativas e homenagens diversas, em 2019 foi a mais assinada. Ja em 2020, quando desenvolvimento social foi a categoria mais assinada, ela caiu para a quarta colocacao e em 2021 foi para quinto lugar, quando a area de frente parlamentar foi a mais assinada.",
        "qas": [
            {
                "question": "Qual foi o número total de assinaturas de projetos propostos em 2021?",
                "id": "9",
                "answers": [
                    {
                        "text": "2164 assinaturas.",
                        "answer_start": 843
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
    {
        "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%.",
        "qas": [
            {
                "question": "Quais categorias tiveram aumentos significativos nas assinaturas em 2021, quando comparadas a 2020?",
                "id": "10",
                "answers": [
                    {
                        "text": "Desenvolvimento social (de 10,14% para 12,66%), saúde e esporte (de 9,15% para 11,41%), proteção dos animais (de 1,12% para 3,10%), e segurança pública (de 0,84% para 2,68%).",
                        "answer_start": 124
                    }
                ],
                 'is_impossible': False
            }
        ]
    },

{
    "context": "Quando comparada com 2017, ano em que esse valor de assinaturas era de 26, considera-se um numero baixo. E sobre os baixos numeros de assinaturas encontrados na area de combate a corrupcao, algumas suposicoes podem ser feitas baseadas tanto no contexto de epoca, assim como no desenrolar dos anos seguintes. Vale destacar que em meados de 2017 o Brasil ainda vivia as atividades da Operacao Lava Jato, que se apresentou como uma operacao que visava combater a corrupcao, sendo que mesma tambem foi uma das principais motivadoras do surgimento de individuos, grupos politicos e movimentos que defendiam o enfrentamento a praticas corruptas na sociedade.",
    "qas": [
        {
            "question": "Como o número de assinaturas na área de combate à corrupção variou entre 2017 e 2021?",
            "id": "11",
            "answers": [
                {
                    "text": "Caiu de 26 em 2017 para 8 em 2021, com variações intermediárias.",
                    "answer_start": 939
                }
            ],
             'is_impossible': False
        }
    ]
},


    {
        "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%.",
        "qas": [
            {
                "question": "Quais foram os temas abordados pelos projetos na área de transparência em 2021?",
                "id": "12",
                "answers": [
                    {
                        "text": "Política de dados abertos, combate a notícias falsas, digitalização de documentos e divulgação de informações relacionadas à Covid-19, vacinação e obras públicas.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
    {
        "context": "De acordo com a CMSP: Grupos suprapartidarios de atuacao vol- tada, em conjunto com a sociedade civil e outros orgaos publicos, ao debate e aprimoramento da legislacao e de politicas publicas no am- bito de sua competencia. Tem tratamento autonomo em relacao a qualquer Comissao Permanente ou Temporaria. Atuam dentro ou fora das dependencias da Camara Municipal, de acordo com seu proposi- to (fonte: https://www.saopaulo.sp.leg.br/atividade-legislativa/frentes- parlamentares/). As 271 assinaturas foram distribuidas em 34 projetos, conforme se verifica no quadro 3, a seguir, que tambem destaca os temas objeto da criacao das frentes parlamentares ao longo do ano de 2021. Note-se que algumas das frentes foram criadas com o mesmo objeto, e que nem todas foram efetivamente transformadas em normas, o que nos leva a algumas reflexoes: - Qual a intencao dos vereadores ao criar tais frentes? - Qual a tramitacao e participacao da sociedade civil? - Existe uma conclusao da Frente Parlamentar, com apresentacao de eventuais resultados obtidos? Outra constatacao interessante (a conferir posteriormente) e que, comparando-se o quadro 1 com o quadro 3, verificam-se muitos projetos de Frente Parlamentar no primeiro ano de mandato. Qual seria uma justificativa para esta incidencia? Quadro - 3 Projetos de Frente Parlamentar Propostos em 2021 Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Fonte: Extraido de https://splegisconsulta.saopaulo.sp.leg.br/Pesquisa/IndexProjeto.",
        "qas": [
            {
                "question": "Quantos projetos de Frente Parlamentar foram propostos em 2021?",
                "id": "13",
                "answers": [
                    {
                        "text": "34 projetos.",
                        "answer_start": 517
                    }
                ],
                 'is_impossible': False
            }
        ]
    },

{
"context": "PR 3/2021 Educacao Basica PR 2/2021 LGBTQIA+ por Direitos e Cidadania. Apoio e Desenvolvimento aos setores Turistico, Hoteleiro, Gastronomico, de Sistemas Expositores, de Eventos, Negocios e de Lazer, e da outras providencias Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Tambem no ano de 2021, em numero absolutos, o total de assinaturas em areas como protecao dos animais e seguranca publica chegam a quase o total feito nas proposicoes que ocorreram na legislatura anterior em ambas. Ja em numeros percentuais, quando comparados com 2020, proporcionalmente as assinaturas na area de protecao dos animais subiram de 1,12% em 2020 para 3,10% do total em 2021; ja em seguranca publica o numero salta de 0,84% em 2020 para 2,68% em 2021. Outras areas que tiveram aumento percentual no total de assinaturas entre 2020 e 2021 foram meio ambiente, habitacao e urbanismo e educacao e cultura que subiram, respectivamente, de 1,33% para 3,00%, de 4,27% para 6,51% e de 8,34% para 9,06%, alem de todas estas categorias terem aumentado em numeros absolutos. Quando se trata de projetos de baixo impacto, as categorias que comumente mais se destacam sao as relacionadas a denominacao de logradouro e a datas comemorativas e homenagens diversas. Na legislatura anterior ja houve anos nos quais estas duas categorias chegaram a receber quase metade do total de assinaturas, sendo 46% do total em 2018 e 43% em 2019. E esta queda no percentual das mesmas no total de assinaturas continuou em 2020, indo para 23%, e posteriormente em 2021 passaram a ser 20,84%. Quando se trata de numeros absolutos, nos anos de 2017, 2018 e 2019, a area de datas comemorativas e homenagens diversas, em 2019 foi a mais assinada. Ja em 2020, quando desenvolvimento social foi a categoria mais assinada, ela caiu para a quarta colocacao e em 2021 foi para quinto lugar, quando a area de frente parlamentar foi a mais assinada.",
        "qas": [
            {
                "question": r"Qual foi a porcentagem de assinaturas atribuída à categoria de saúde e esporte em 2021?",
                "id": "14",
                "answers": [
                    {
                        "text": "As planilhas com as categorias foram enviadas para os vereadores de São Paulo (incluindo os suplentes que chegaram a ser empossados em algum momento do ano e assinaram projetos legislativos) para confirmação das informações. Entretanto, o retorno a estas solicitações foi baixíssimo, com somente cinco legisladores tendo dado respostas dentro do prazo estipulado.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
        {
        "context": "PR 3/2021 Educacao Basica PR 2/2021 LGBTQIA+ por Direitos e Cidadania. Apoio e Desenvolvimento aos setores Turistico, Hoteleiro, Gastronomico, de Sistemas Expositores, de Eventos, Negocios e de Lazer, e da outras providencias Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Tambem no ano de 2021, em numero absolutos, o total de assinaturas em areas como protecao dos animais e seguranca publica chegam a quase o total feito nas proposicoes que ocorreram na legislatura anterior em ambas. Ja em numeros percentuais, quando comparados com 2020, proporcionalmente as assinaturas na area de protecao dos animais subiram de 1,12% em 2020 para 3,10% do total em 2021; ja em seguranca publica o numero salta de 0,84% em 2020 para 2,68% em 2021. Outras areas que tiveram aumento percentual no total de assinaturas entre 2020 e 2021 foram meio ambiente, habitacao e urbanismo e educacao e cultura que subiram, respectivamente, de 1,33% para 3,00%, de 4,27% para 6,51% e de 8,34% para 9,06%, alem de todas estas categorias terem aumentado em numeros absolutos. Quando se trata de projetos de baixo impacto, as categorias que comumente mais se destacam sao as relacionadas a denominacao de logradouro e a datas comemorativas e homenagens diversas. Na legislatura anterior ja houve anos nos quais estas duas categorias chegaram a receber quase metade do total de assinaturas, sendo 46% do total em 2018 e 43% em 2019. E esta queda no percentual das mesmas no total de assinaturas continuou em 2020, indo para 23%, e posteriormente em 2021 passaram a ser 20,84%. Quando se trata de numeros absolutos, nos anos de 2017, 2018 e 2019, a area de datas comemorativas e homenagens diversas, em 2019 foi a mais assinada. Ja em 2020, quando desenvolvimento social foi a categoria mais assinada, ela caiu para a quarta colocacao e em 2021 foi para quinto lugar, quando a area de frente parlamentar foi a mais assinada.",
        "qas": [
            {
                "question": r"Quais categorias tiveram queda proporcional no total de assinaturas em 2021?",
                "id": "15",
                "answers": [
                    {
                        "text": "Denominação de logradouro e datas comemorativas e homenagens diversas.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
        {
        "context": " Por fim, outro ponto a destacar sao as assinaturas em projetos nas categorias de combate a corrupcao e transparencia. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br No tocante aos projetos de lei na area de combate a corrupcao a quantidade de assinaturas permaneceu baixa em 2021, com um total de 8 assinaturas, e os projetos nesta area giraram em torno da obrigatoriedade da prestacao de informacoes ao legislativo sobre alteracoes das tarifas do servico transporte publico de passageiros do municipio, da retomada da gestao administrativa e pedagogica de equipamentos repassados a gestao indireta por associacoes e OSCIPs para o provimento da educacao infantil de 0 a 3 anos de idade e da politica municipal de fiscalizacao, prevencao e combate a adulteracao e comercializacao de combustiveis adulterados.",
        "qas": [
            {
                "question": r"Quais são as consequências da baixa transparência mencionadas no relatório?",
                "id": "16",
                "answers": [
                    {
                        "text": "Dificulta a elaboração e avaliação de políticas públicas, o combate à corrupção e inviabiliza o controle social.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
        {
        "context": "Quando comparada com 2017, ano em que esse valor de assinaturas era de 26, considera-se um numero baixo. E sobre os baixos numeros de assinaturas encontrados na area de combate a corrupcao, algumas suposicoes podem ser feitas baseadas tanto no contexto de epoca, assim como no desenrolar dos anos seguintes. Vale destacar que em meados de 2017 o Brasil ainda vivia as atividades da Operacao Lava Jato, que se apresentou como uma operacao que visava combater a corrupcao, sendo que mesma tambem foi uma das principais motivadoras do surgimento de individuos, grupos politicos e movimentos que defendiam o enfrentamento a praticas corruptas na sociedade. No entanto, o que se observou nos anos seguintes foi o progressivo esvaziamento desta causa, o que leva a supor que a bandeira do combate a corrupcao nao passou de mera retorica que serviu para promover os projetos de poder de alguns setores da sociedade. E isto nao foi diferente na Camara dos Vereadores de Sao Paulo, porque no ano de 2018 nao houve nenhuma assinatura em projetos para esta area, em 2019 este numero sobe para 1, em 2020 aumenta para 9 e em 2021 cai para 8. O dever de agora e observar se o numero de assinaturas em projetos de combate a corrupcao vai aumentar ou se vai estagnar ou diminuir. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Ja o numero de assinaturas em projetos de lei na area de transparencia no ano de 2021 foi de 23 ao todo.",
        "qas": [
            {
                "question": r"Como o relatório relaciona a Operação Lava Jato à redução de assinaturas em projetos de combate à corrupção?",
                "id": "17",
                "answers": [
                    {
                        "text": "Sugere que o esvaziamento do tema reflete a perda de prioridade política, antes impulsionada pela Lava Jato.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
        {
        "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%. Em seguida aparecem os projetos na area de frente parlamentar foram as mais assinadas, com 271 assinaturas, representando 12,52% do total.",
        "qas": [
            {
                "question": r"O que o relatório destaca como um ponto positivo na atuação legislativa em 2021?",
                "id": "18",
                "answers": [
                    {
                        "text": "O aumento de assinaturas em áreas de impacto social, como desenvolvimento social e saúde e esporte.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
    {
        "context": "De acordo com a CMSP: Grupos suprapartidarios de atuacao vol- tada, em conjunto com a sociedade civil e outros orgaos publicos, ao debate e aprimoramento da legislacao e de politicas publicas no am- bito de sua competencia. Tem tratamento autonomo em relacao a qualquer Comissao Permanente ou Temporaria. Atuam dentro ou fora das dependencias da Camara Municipal, de acordo com seu proposi- to (fonte: https://www.saopaulo.sp.leg.br/atividade-legislativa/frentes- parlamentares/). As 271 assinaturas foram distribuidas em 34 projetos, conforme se verifica no quadro 3, a seguir, que tambem destaca os temas objeto da criacao das frentes parlamentares ao longo do ano de 2021. Note-se que algumas das frentes foram criadas com o mesmo objeto, e que nem todas foram efetivamente transformadas em normas, o que nos leva a algumas reflexoes: - Qual a intencao dos vereadores ao criar tais frentes? - Qual a tramitacao e participacao da sociedade civil? - Existe uma conclusao da Frente Parlamentar, com apresentacao de eventuais resultados obtidos? Outra constatacao interessante (a conferir posteriormente) e que, comparando-se o quadro 1 com o quadro 3, verificam-se muitos projetos de Frente Parlamentar no primeiro ano de mandato. Qual seria uma justificativa para esta incidencia? Quadro - 3 Projetos de Frente Parlamentar Propostos em 2021 Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Fonte: Extraido de https://splegisconsulta.saopaulo.sp.leg.br/Pesquisa/IndexProjeto Quanto aos projetos propostos em outras areas algumas informacoes chamam a atencao, sendo que a primeira delas e o fato de que na area de Saude e Esporte o total de assinaturas subiu de 219 para 247, o que proporcionalmente corresponde a 11,41% do total de projetos propostos em 2021 e representando um aumento de 2,26% quando comparado a 2020. Provavelmente esta subida de valores pode ter sido influenciada pela pandemia de Covid-19. Projeto Tema Norma PR 54/2021 Servico Publico Municipal PR 53/2021 Desenvolvimento e Atividade Economica da Zona Leste da Cidade de Sao Paulo 16/2021 PR 51/2021 Defesa da Mobilidade Cicloviaria e da Bicicleta. 18/2021 PR 50/2021 Contra a Reforma da Previdencia Municipal BSAMPAPREV II PR 47/2021 Defesa do Saneamento, residuos solidos e meio ambiente. PR 46/2021 Juventude na Cidade de Sao Paulo PR 45/2021 Defesa dos Direitos da Pessoa com Deficiencia e Doencas Raras. 15/2021 PR 44/2021 Apoio as Bandas e Fanfarras da Cidade de Sao Paulo. PR 43/2021 Defesa da implantacao definitiva do campus da Unifesp Zona Leste PR 40/2021 Combate ao Suicidio e Autolesao e da outras providenciasB PR 39/2021 Dispoe sobre a criacao da Frente Parlamentar das Mudancas Climaticas no Municipio de Sao Paulo. PR 36/2021 Defesa do esporte e do futebol de varzea na Cidade de Sao Paulo. 17/2021 PR 34/2021 Defesa da Adocao, do Acolhimento Familiar PR 32/2021 Ambientalista. 8/2021 PR 29/2021 PR 26/2021 Cooperacao e Interlocucao das Associacoes e Entidades de Fomento ao Empreendedorismo na Cidade de Sao Paulo. 14/2021 PR 25/2021 Defesa do Setor de Entretenimento, Turismo, Esporte, Cultura e Eventos em Geral da Cidade de Sao Paulo. 13/2021 PR 23/2021 Infraestrutura e Engenharia de Sao Paulo 11/2021 PR 21/2021 Defesa da Vida, da Familia e do Direito Natural. 5/2021 PR 20/2021 integracao Cidade de Sao Paulo e Cuba PR 19/2021 Comercio, Empreendedorismo e Servicos da cidade de Sao Paulo PR 18/2021 Apoio a Cultura e a Educacao, diante das dificuldades ocasionadas pela pandemia PR 17/2021 Mista de Ciencia, Tecnologia, Pesquisa e Inovacao no Municipio de Sao Paulo PR 15/2021 Apoio a Cultura e a Educacao, diante das dificuldades ocasionadas pela pandemia PR 12/2021 Combate a pandemia da COVID-19 PR 11/2021 Combate ao Racismo PR 9/2021 Combate a fome. 3/2021 PR 8/2021 Fortalecimento do Terceiro Setor PR 7/2021 Combate e Prevencao ao Suicidio e Autolesao PR 6/2021 Defesa dos direitos da pessoa com transtorno do espectro autista 7/2021 PR 5/2021 Frente Parlamentar Crista em Defesa da Familia PR 4/2021 Fiscalizacao, Acompanhamento e controle do processo de evolucao da vacinacao contra a Covid-19 na cidade de Sao Paulo. PR 3/2021 Educacao Basica PR 2/2021 LGBTQIA+ por Direitos e Cidadania. Apoio e Desenvolvimento aos setores Turistico, Hoteleiro, Gastronomico, de Sistemas Expositores, de Eventos, Negocios e de Lazer, e da outras providencias Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Tambem no ano de 2021, em numero absolutos, o total de assinaturas em areas como protecao dos animais e seguranca publica chegam a quase o total feito nas proposicoes que ocorreram na legislatura anterior em ambas. Ja em numeros percentuais, quando comparados com 2020, proporcionalmente as assinaturas na area de protecao dos animais subiram de 1,12% em 2020 para 3,10% do total em 2021; ja em seguranca publica o numero salta de 0,84% em 2020 para 2,68% em 2021. Outras areas que tiveram aumento percentual no total de assinaturas entre 2020 e 2021 foram meio ambiente, habitacao e urbanismo e educacao e cultura que subiram, respectivamente, de 1,33% para 3,00%, de 4,27% para 6,51% e de 8,34% para 9,06%, alem de todas estas categorias terem aumentado em numeros absolutos. Quando se trata de projetos de baixo impacto, as categorias que comumente mais se destacam sao as relacionadas a denominacao de logradouro e a datas comemorativas e homenagens diversas. Na legislatura anterior ja houve anos nos quais estas duas categorias chegaram a receber quase metade do total de assinaturas, sendo 46% do total em 2018 e 43% em 2019. E esta queda no percentual das mesmas no total de assinaturas continuou em 2020, indo para 23%, e posteriormente em 2021 passaram a ser 20,84%. Quando se trata de numeros absolutos, nos anos de 2017, 2018 e 2019, a area de datas comemorativas e homenagens diversas, em 2019 foi a mais assinada. Ja em 2020, quando desenvolvimento social foi a categoria mais assinada, ela caiu para a quarta colocacao e em 2021 foi para quinto lugar, quando a area de frente parlamentar foi a mais assinada. Por fim, outro ponto a destacar sao as assinaturas em projetos nas categorias de combate a corrupcao e transparencia. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br No tocante aos projetos de lei na area de combate a corrupcao a quantidade de assinaturas permaneceu baixa em 2021, com um total de 8 assinaturas, e os projetos nesta area giraram em torno da obrigatoriedade da prestacao de informacoes ao legislativo sobre alteracoes das tarifas do servico transporte publico de passageiros do municipio, da retomada da gestao administrativa e pedagogica de equipamentos repassados a gestao indireta por associacoes e OSCIPs para o provimento da educacao infantil de 0 a 3 anos de idade e da politica municipal de fiscalizacao, prevencao e combate a adulteracao e comercializacao de combustiveis adulterados. Quando comparada com 2017, ano em que esse valor de assinaturas era de 26, considera-se um numero baixo. E sobre os baixos numeros de assinaturas encontrados na area de combate a corrupcao, algumas suposicoes podem ser feitas baseadas tanto no contexto de epoca, assim como no desenrolar dos anos seguintes. Vale destacar que em meados de 2017 o Brasil ainda vivia as atividades da Operacao Lava Jato, que se apresentou como uma operacao que visava combater a corrupcao, sendo que mesma tambem foi uma das principais motivadoras do surgimento de individuos, grupos politicos e movimentos que defendiam o enfrentamento a praticas corruptas na sociedade. No entanto, o que se observou nos anos seguintes foi o progressivo esvaziamento desta causa, o que leva a supor que a bandeira do combate a corrupcao nao passou de mera retorica que serviu para promover os projetos de poder de alguns setores da sociedade. E isto nao foi diferente na Camara dos Vereadores de Sao Paulo, porque no ano de 2018 nao houve nenhuma assinatura em projetos para esta area, em 2019 este numero sobe para 1, em 2020 aumenta para 9 e em 2021 cai para 8. O dever de agora e observar se o numero de assinaturas em projetos de combate a corrupcao vai aumentar ou se vai estagnar ou diminuir. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Ja o numero de assinaturas em projetos de lei na area de transparencia no ano de 2021 foi de 23 ao todo. Quando comparado com a sequencia historica, que surgiu em 2017, este chega a ser um numero alto para a area, ficando atras apenas do ano de 2019 quando houve 35 assinaturas ao todo. Na area de transparencia os projetos giraram em torno da consolidacao da politica de dados abertos e transparencia no ambito do municipio, combate a noticias falsas, digitalizacao de documentos fisicos do municipio, a criacao de um observatorio de dados abertos do municipio e divulgacao de informacoes relacionadas a obras, contaminacao por covid-19, vacinacao, contratos de terceirizacao de servicos e os processos administrativos relacionados a regularizacao fundiaria. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br CONSIDERACOES FINAIS E valido e importante destacar que o exercicio do controle social e extremamente necessario para saber se as instituicoes estao em bom funcionamento e se mantem sintonia com as demandas e a realidade da sociedade, alem de ajudar a sociedade a propor caminhos para melhorar a gestao publica, a fomentar a participacao social na administracao publica e a levar conhecimento ao publico. A principio, nota-se que a Camara dos Vereadores no ano de 2021 esteve mais ativa na questao de proposicao de projetos legislativos que nos anos anteriores. Alem disso, pelo segundo ano os projetos propostos em areas de grande impacto permaneceram como as mais assinadas, a saber: desenvolvimento social e saude e esporte. Isto e muito positivo. No entanto e importante dar atencao para saber se isto e apenas uma questao mais relacionada a pandemia ou se realmente o poder legislativo municipal esta comecando a dar atencao a areas de grande importancia social, pois as areas de denominacao de logradouro e datas comemorativas e homenagens diversas ainda permanecem como algumas das categorias mais assinadas na Camara. Ja o ponto negativo esta a baixa assinatura em projetos de transparencia e de combate a corrupcao. E de grande importancia que os legisladores deem mais atencao a estas areas que possuem uma dependencia mutua, pois a baixa transparencia torna mais dificil a elaboracao e a avaliacao de politicas publicas, dificulta o combate a corrupcao e inviabiliza o controle social do poder publico pela sociedade.",
        "qas": [
            {
                "question": r"Que sugestões o relatório oferece para melhorar o controle social no contexto legislativo?",
                "id": "19",
                "answers": [
                    {
                        "text": " Maior participação social, estímulo à transparência e engajamento na avaliação das ações legislativas.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
        {
        "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%. Em seguida aparecem os projetos na area de frente parlamentar foram as mais assinadas, com 271 assinaturas, representando 12,52% do total.",
                "qas": [
            {
                "question": r"Quais foram as mudanças de destaque nas categorias de desenvolvimento social e educação entre 2020 e 2021?",
                "id": "20",
                "answers": [
                    {
                        "text": "Desenvolvimento social aumentou de 10,14% para 12,66%, e educação subiu de 8,34% para 9,06%.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
{
        "context": " Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%. Em seguida aparecem os projetos na area de frente parlamentar foram as mais assinadas, com 271 assinaturas, representando 12,52% do total.",
        "qas": [
            {
                "question": r"Qual foi o método utilizado para classificar os projetos propostos pelos vereadores?",
                "id": "21",
                "answers": [
                    {
                        "text": "As planilhas individuais foram classificadas em categorias pré-estabelecidas pela equipe.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
        {
        "context": "Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br RELATORIO DA ANALISE DE PROJETOS PROPOSTOS NO ANO DE 20211 INTRODUCAO O grupo de monitoramento do legislativo do Observatorio Social do Brasil de Sao Paulo realizou o levantamento e a analise da atividade de projetos propostos do ano de 2021, de forma igual ao que vinha realizando desde o ano de 2017. Como metodologia para a realizacao da atividade em 2021, foram distribuidas entre os voluntarios planilhas individuais dos vereadores, para o levantamento dos projetos propostos (Fonte: https://splegisconsulta.saopaulo.sp.leg.br/) e sua classificacao em categorias, pre-estabelecidas pela equipe. Tendo em vista que as categorias trazem um certo grau de subjetividade, apos os dados serem levantados, as planilhas foram enviadas para os vereadores de Sao Paulo (Incluindo os suplentes que chegaram a ser empossados em algum momento do ano e assinaram projetos legislativos) para confirmacao das informacoes. Entretanto o retorno a estas solicitacoes foi baixissimo, com somente cinco legisladores tendo dado respostas dentro do prazo estipulado, denotando falta de interesse dos parlamentares em atender a pedidos de informacoes da sociedade civil organizada. A analise a seguir, portanto, levou em consideracao todas as demais informacoes levantadas como devidamente confirmadas, com as planilhas individuais dos vereadores, a planilha geral estando presente em 1 Errata: O total de assinaturas de projetos propostos e de 2164 e nao de 2176 como apontava o relatorio anteriormente. Tambem a area com mais assinaturas e a de desenvolvimento social e nao frente parlamentar, que na verdade e a segunda mais assinada. As correcoes foram possiveis devido a montagem do painel de projetos propostos, que permitiu maior precisao na contagem dos dados quantitativos na atividade e sera publicado em breve.",
        "qas": [
            {
                "question": r"Por que o relatório menciona que o retorno dos vereadores às solicitações foi baixo?",
                "id": "22",
                "answers": [
                    {
                        "text": "Apenas cinco legisladores responderam no prazo estipulado, indicando falta de interesse em colaborar com a sociedade civil organizada.",
                        "answer_start": 9
                    }
                ],
                 'is_impossible': False
            }
        ]
    },
{
        "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%. Em seguida aparecem os projetos na area de frente parlamentar foram as mais assinadas, com 271 assinaturas, representando 12,52% do total.",
        "qas": [
            {
                "question": r"Como o relatório avalia a queda de assinaturas em projetos relacionados a denominação de logradouro?",
                "id": "23",
                "answers": [
                    {
                        "text": "Apesar de representarem menos assinaturas proporcionalmente em 2021 (20,84%), continuam sendo uma das categorias mais assinadas.",
                        "answer_start": 9
                    }
                ]
            }
        ]
    },
        {
        "context": " Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%. Em seguida aparecem os projetos na area de frente parlamentar foram as mais assinadas, com 271 assinaturas, representando 12,52% do total.",
        "qas": [
            {
                "question": r"Quais foram as áreas com maior aumento proporcional de assinaturas entre 2020 e 2021?",
                "id": "24",
                'is_impossible': False,
                "answers": [
                    {
                        "text": "Meio ambiente (1,33% para 3,00%), habitação e urbanismo (4,27% para 6,51%), e educação e cultura (8,34% para 9,06%).",
                        "answer_start": 9
                    }
                ]
            }
        ]
    },
{
        "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%. Em seguida aparecem os projetos na area de frente parlamentar foram as mais assinadas, com 271 assinaturas, representando 12,52% do total.",
        "qas": [
            {
                "question": r"Como o relatório justifica a baixa quantidade de assinaturas em projetos de combate à corrupção em 2021?",
                "id": "25",
                'is_impossible': False,
                "answers": [
                    {
                        "text": "Argumenta que houve um esvaziamento da pauta após a Operação Lava Jato, indicando que o tema pode ter sido usado de forma retórica no passado.",
                        "answer_start": 9
                    }
                ]
            }
        ]
    }]
test_data=[
        {
        "context": "De acordo com a CMSP: Grupos suprapartidarios de atuacao vol- tada, em conjunto com a sociedade civil e outros orgaos publicos, ao debate e aprimoramento da legislacao e de politicas publicas no am- bito de sua competencia. Tem tratamento autonomo em relacao a qualquer Comissao Permanente ou Temporaria. Atuam dentro ou fora das dependencias da Camara Municipal, de acordo com seu proposi- to (fonte: https://www.saopaulo.sp.leg.br/atividade-legislativa/frentes- parlamentares/). As 271 assinaturas foram distribuidas em 34 projetos, conforme se verifica no quadro 3, a seguir, que tambem destaca os temas objeto da criacao das frentes parlamentares ao longo do ano de 2021. Note-se que algumas das frentes foram criadas com o mesmo objeto, e que nem todas foram efetivamente transformadas em normas, o que nos leva a algumas reflexoes: - Qual a intencao dos vereadores ao criar tais frentes? - Qual a tramitacao e participacao da sociedade civil? - Existe uma conclusao da Frente Parlamentar, com apresentacao de eventuais resultados obtidos? Outra constatacao interessante (a conferir posteriormente) e que, comparando-se o quadro 1 com o quadro 3, verificam-se muitos projetos de Frente Parlamentar no primeiro ano de mandato. Qual seria uma justificativa para esta incidencia? Quadro - 3 Projetos de Frente Parlamentar Propostos em 2021 Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Fonte: Extraido de https://splegisconsulta.saopaulo.sp.leg.br/Pesquisa/IndexProjeto Quanto aos projetos propostos em outras areas algumas informacoes chamam a atencao, sendo que a primeira delas e o fato de que na area de Saude e Esporte o total de assinaturas subiu de 219 para 247, o que proporcionalmente corresponde a 11,41% do total de projetos propostos em 2021 e representando um aumento de 2,26% quando comparado a 2020. Provavelmente esta subida de valores pode ter sido influenciada pela pandemia de Covid-19. Projeto Tema Norma PR 54/2021 Servico Publico Municipal PR 53/2021 Desenvolvimento e Atividade Economica da Zona Leste da Cidade de Sao Paulo 16/2021 PR 51/2021 Defesa da Mobilidade Cicloviaria e da Bicicleta. 18/2021 PR 50/2021 Contra a Reforma da Previdencia Municipal BSAMPAPREV II PR 47/2021 Defesa do Saneamento, residuos solidos e meio ambiente. PR 46/2021 Juventude na Cidade de Sao Paulo PR 45/2021 Defesa dos Direitos da Pessoa com Deficiencia e Doencas Raras. 15/2021 PR 44/2021 Apoio as Bandas e Fanfarras da Cidade de Sao Paulo. PR 43/2021 Defesa da implantacao definitiva do campus da Unifesp Zona Leste PR 40/2021 Combate ao Suicidio e Autolesao e da outras providenciasB PR 39/2021 Dispoe sobre a criacao da Frente Parlamentar das Mudancas Climaticas no Municipio de Sao Paulo. PR 36/2021 Defesa do esporte e do futebol de varzea na Cidade de Sao Paulo. 17/2021 PR 34/2021 Defesa da Adocao, do Acolhimento Familiar PR 32/2021 Ambientalista. 8/2021 PR 29/2021 PR 26/2021 Cooperacao e Interlocucao das Associacoes e Entidades de Fomento ao Empreendedorismo na Cidade de Sao Paulo. 14/2021 PR 25/2021 Defesa do Setor de Entretenimento, Turismo, Esporte, Cultura e Eventos em Geral da Cidade de Sao Paulo. 13/2021 PR 23/2021 Infraestrutura e Engenharia de Sao Paulo 11/2021 PR 21/2021 Defesa da Vida, da Familia e do Direito Natural. 5/2021 PR 20/2021 integracao Cidade de Sao Paulo e Cuba PR 19/2021 Comercio, Empreendedorismo e Servicos da cidade de Sao Paulo PR 18/2021 Apoio a Cultura e a Educacao, diante das dificuldades ocasionadas pela pandemia PR 17/2021 Mista de Ciencia, Tecnologia, Pesquisa e Inovacao no Municipio de Sao Paulo PR 15/2021 Apoio a Cultura e a Educacao, diante das dificuldades ocasionadas pela pandemia PR 12/2021 Combate a pandemia da COVID-19 PR 11/2021 Combate ao Racismo PR 9/2021 Combate a fome. 3/2021 PR 8/2021 Fortalecimento do Terceiro Setor PR 7/2021 Combate e Prevencao ao Suicidio e Autolesao PR 6/2021 Defesa dos direitos da pessoa com transtorno do espectro autista 7/2021 PR 5/2021 Frente Parlamentar Crista em Defesa da Familia PR 4/2021 Fiscalizacao, Acompanhamento e controle do processo de evolucao da vacinacao contra a Covid-19 na cidade de Sao Paulo. PR 3/2021 Educacao Basica PR 2/2021 LGBTQIA+ por Direitos e Cidadania. Apoio e Desenvolvimento aos setores Turistico, Hoteleiro, Gastronomico, de Sistemas Expositores, de Eventos, Negocios e de Lazer, e da outras providencias Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Tambem no ano de 2021, em numero absolutos, o total de assinaturas em areas como protecao dos animais e seguranca publica chegam a quase o total feito nas proposicoes que ocorreram na legislatura anterior em ambas. Ja em numeros percentuais, quando comparados com 2020, proporcionalmente as assinaturas na area de protecao dos animais subiram de 1,12% em 2020 para 3,10% do total em 2021; ja em seguranca publica o numero salta de 0,84% em 2020 para 2,68% em 2021. Outras areas que tiveram aumento percentual no total de assinaturas entre 2020 e 2021 foram meio ambiente, habitacao e urbanismo e educacao e cultura que subiram, respectivamente, de 1,33% para 3,00%, de 4,27% para 6,51% e de 8,34% para 9,06%, alem de todas estas categorias terem aumentado em numeros absolutos. Quando se trata de projetos de baixo impacto, as categorias que comumente mais se destacam sao as relacionadas a denominacao de logradouro e a datas comemorativas e homenagens diversas. Na legislatura anterior ja houve anos nos quais estas duas categorias chegaram a receber quase metade do total de assinaturas, sendo 46% do total em 2018 e 43% em 2019. E esta queda no percentual das mesmas no total de assinaturas continuou em 2020, indo para 23%, e posteriormente em 2021 passaram a ser 20,84%. Quando se trata de numeros absolutos, nos anos de 2017, 2018 e 2019, a area de datas comemorativas e homenagens diversas, em 2019 foi a mais assinada. Ja em 2020, quando desenvolvimento social foi a categoria mais assinada, ela caiu para a quarta colocacao e em 2021 foi para quinto lugar, quando a area de frente parlamentar foi a mais assinada. Por fim, outro ponto a destacar sao as assinaturas em projetos nas categorias de combate a corrupcao e transparencia. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br No tocante aos projetos de lei na area de combate a corrupcao a quantidade de assinaturas permaneceu baixa em 2021, com um total de 8 assinaturas, e os projetos nesta area giraram em torno da obrigatoriedade da prestacao de informacoes ao legislativo sobre alteracoes das tarifas do servico transporte publico de passageiros do municipio, da retomada da gestao administrativa e pedagogica de equipamentos repassados a gestao indireta por associacoes e OSCIPs para o provimento da educacao infantil de 0 a 3 anos de idade e da politica municipal de fiscalizacao, prevencao e combate a adulteracao e comercializacao de combustiveis adulterados. Quando comparada com 2017, ano em que esse valor de assinaturas era de 26, considera-se um numero baixo. E sobre os baixos numeros de assinaturas encontrados na area de combate a corrupcao, algumas suposicoes podem ser feitas baseadas tanto no contexto de epoca, assim como no desenrolar dos anos seguintes. Vale destacar que em meados de 2017 o Brasil ainda vivia as atividades da Operacao Lava Jato, que se apresentou como uma operacao que visava combater a corrupcao, sendo que mesma tambem foi uma das principais motivadoras do surgimento de individuos, grupos politicos e movimentos que defendiam o enfrentamento a praticas corruptas na sociedade. No entanto, o que se observou nos anos seguintes foi o progressivo esvaziamento desta causa, o que leva a supor que a bandeira do combate a corrupcao nao passou de mera retorica que serviu para promover os projetos de poder de alguns setores da sociedade. E isto nao foi diferente na Camara dos Vereadores de Sao Paulo, porque no ano de 2018 nao houve nenhuma assinatura em projetos para esta area, em 2019 este numero sobe para 1, em 2020 aumenta para 9 e em 2021 cai para 8. O dever de agora e observar se o numero de assinaturas em projetos de combate a corrupcao vai aumentar ou se vai estagnar ou diminuir. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Ja o numero de assinaturas em projetos de lei na area de transparencia no ano de 2021 foi de 23 ao todo. Quando comparado com a sequencia historica, que surgiu em 2017, este chega a ser um numero alto para a area, ficando atras apenas do ano de 2019 quando houve 35 assinaturas ao todo. Na area de transparencia os projetos giraram em torno da consolidacao da politica de dados abertos e transparencia no ambito do municipio, combate a noticias falsas, digitalizacao de documentos fisicos do municipio, a criacao de um observatorio de dados abertos do municipio e divulgacao de informacoes relacionadas a obras, contaminacao por covid-19, vacinacao, contratos de terceirizacao de servicos e os processos administrativos relacionados a regularizacao fundiaria. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br CONSIDERACOES FINAIS E valido e importante destacar que o exercicio do controle social e extremamente necessario para saber se as instituicoes estao em bom funcionamento e se mantem sintonia com as demandas e a realidade da sociedade, alem de ajudar a sociedade a propor caminhos para melhorar a gestao publica, a fomentar a participacao social na administracao publica e a levar conhecimento ao publico. A principio, nota-se que a Camara dos Vereadores no ano de 2021 esteve mais ativa na questao de proposicao de projetos legislativos que nos anos anteriores. Alem disso, pelo segundo ano os projetos propostos em areas de grande impacto permaneceram como as mais assinadas, a saber: desenvolvimento social e saude e esporte. Isto e muito positivo. No entanto e importante dar atencao para saber se isto e apenas uma questao mais relacionada a pandemia ou se realmente o poder legislativo municipal esta comecando a dar atencao a areas de grande importancia social, pois as areas de denominacao de logradouro e datas comemorativas e homenagens diversas ainda permanecem como algumas das categorias mais assinadas na Camara. Ja o ponto negativo esta a baixa assinatura em projetos de transparencia e de combate a corrupcao. E de grande importancia que os legisladores deem mais atencao a estas areas que possuem uma dependencia mutua, pois a baixa transparencia torna mais dificil a elaboracao e a avaliacao de politicas publicas, dificulta o combate a corrupcao e inviabiliza o controle social do poder publico pela sociedade.",
        "qas": [
            {
                "question": r"Qual foi o impacto da pandemia de Covid-19 nas assinaturas de projetos em 2021?",
                "id": "86",

                "answers": [
                    {
                        "text": "A pandemia influenciou diretamente projetos em áreas como saúde e esporte, proteção dos animais e educação.",
                        "answer_start": 30
                    }
                ],
                  'is_impossible': False,
           }
        ]
    },
{
        "context": "De acordo com a CMSP: Grupos suprapartidarios de atuacao vol- tada, em conjunto com a sociedade civil e outros orgaos publicos, ao debate e aprimoramento da legislacao e de politicas publicas no am- bito de sua competencia. Tem tratamento autonomo em relacao a qualquer Comissao Permanente ou Temporaria. Atuam dentro ou fora das dependencias da Camara Municipal, de acordo com seu proposi- to (fonte: https://www.saopaulo.sp.leg.br/atividade-legislativa/frentes- parlamentares/). As 271 assinaturas foram distribuidas em 34 projetos, conforme se verifica no quadro 3, a seguir, que tambem destaca os temas objeto da criacao das frentes parlamentares ao longo do ano de 2021. Note-se que algumas das frentes foram criadas com o mesmo objeto, e que nem todas foram efetivamente transformadas em normas, o que nos leva a algumas reflexoes: - Qual a intencao dos vereadores ao criar tais frentes? - Qual a tramitacao e participacao da sociedade civil? - Existe uma conclusao da Frente Parlamentar, com apresentacao de eventuais resultados obtidos? Outra constatacao interessante (a conferir posteriormente) e que, comparando-se o quadro 1 com o quadro 3, verificam-se muitos projetos de Frente Parlamentar no primeiro ano de mandato. Qual seria uma justificativa para esta incidencia? Quadro - 3 Projetos de Frente Parlamentar Propostos em 2021 Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Fonte: Extraido de https://splegisconsulta.saopaulo.sp.leg.br/Pesquisa/IndexProjeto Quanto aos projetos propostos em outras areas algumas informacoes chamam a atencao, sendo que a primeira delas e o fato de que na area de Saude e Esporte o total de assinaturas subiu de 219 para 247, o que proporcionalmente corresponde a 11,41% do total de projetos propostos em 2021 e representando um aumento de 2,26% quando comparado a 2020. Provavelmente esta subida de valores pode ter sido influenciada pela pandemia de Covid-19. Projeto Tema Norma PR 54/2021 Servico Publico Municipal PR 53/2021 Desenvolvimento e Atividade Economica da Zona Leste da Cidade de Sao Paulo 16/2021 PR 51/2021 Defesa da Mobilidade Cicloviaria e da Bicicleta. 18/2021 PR 50/2021 Contra a Reforma da Previdencia Municipal BSAMPAPREV II PR 47/2021 Defesa do Saneamento, residuos solidos e meio ambiente. PR 46/2021 Juventude na Cidade de Sao Paulo PR 45/2021 Defesa dos Direitos da Pessoa com Deficiencia e Doencas Raras. 15/2021 PR 44/2021 Apoio as Bandas e Fanfarras da Cidade de Sao Paulo. PR 43/2021 Defesa da implantacao definitiva do campus da Unifesp Zona Leste PR 40/2021 Combate ao Suicidio e Autolesao e da outras providenciasB PR 39/2021 Dispoe sobre a criacao da Frente Parlamentar das Mudancas Climaticas no Municipio de Sao Paulo. PR 36/2021 Defesa do esporte e do futebol de varzea na Cidade de Sao Paulo. 17/2021 PR 34/2021 Defesa da Adocao, do Acolhimento Familiar PR 32/2021 Ambientalista. 8/2021 PR 29/2021 PR 26/2021 Cooperacao e Interlocucao das Associacoes e Entidades de Fomento ao Empreendedorismo na Cidade de Sao Paulo. 14/2021 PR 25/2021 Defesa do Setor de Entretenimento, Turismo, Esporte, Cultura e Eventos em Geral da Cidade de Sao Paulo. 13/2021 PR 23/2021 Infraestrutura e Engenharia de Sao Paulo 11/2021 PR 21/2021 Defesa da Vida, da Familia e do Direito Natural. 5/2021 PR 20/2021 integracao Cidade de Sao Paulo e Cuba PR 19/2021 Comercio, Empreendedorismo e Servicos da cidade de Sao Paulo PR 18/2021 Apoio a Cultura e a Educacao, diante das dificuldades ocasionadas pela pandemia PR 17/2021 Mista de Ciencia, Tecnologia, Pesquisa e Inovacao no Municipio de Sao Paulo PR 15/2021 Apoio a Cultura e a Educacao, diante das dificuldades ocasionadas pela pandemia PR 12/2021 Combate a pandemia da COVID-19 PR 11/2021 Combate ao Racismo PR 9/2021 Combate a fome. 3/2021 PR 8/2021 Fortalecimento do Terceiro Setor PR 7/2021 Combate e Prevencao ao Suicidio e Autolesao PR 6/2021 Defesa dos direitos da pessoa com transtorno do espectro autista 7/2021 PR 5/2021 Frente Parlamentar Crista em Defesa da Familia PR 4/2021 Fiscalizacao, Acompanhamento e controle do processo de evolucao da vacinacao contra a Covid-19 na cidade de Sao Paulo. PR 3/2021 Educacao Basica PR 2/2021 LGBTQIA+ por Direitos e Cidadania. Apoio e Desenvolvimento aos setores Turistico, Hoteleiro, Gastronomico, de Sistemas Expositores, de Eventos, Negocios e de Lazer, e da outras providencias Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Tambem no ano de 2021, em numero absolutos, o total de assinaturas em areas como protecao dos animais e seguranca publica chegam a quase o total feito nas proposicoes que ocorreram na legislatura anterior em ambas. Ja em numeros percentuais, quando comparados com 2020, proporcionalmente as assinaturas na area de protecao dos animais subiram de 1,12% em 2020 para 3,10% do total em 2021; ja em seguranca publica o numero salta de 0,84% em 2020 para 2,68% em 2021. Outras areas que tiveram aumento percentual no total de assinaturas entre 2020 e 2021 foram meio ambiente, habitacao e urbanismo e educacao e cultura que subiram, respectivamente, de 1,33% para 3,00%, de 4,27% para 6,51% e de 8,34% para 9,06%, alem de todas estas categorias terem aumentado em numeros absolutos. Quando se trata de projetos de baixo impacto, as categorias que comumente mais se destacam sao as relacionadas a denominacao de logradouro e a datas comemorativas e homenagens diversas. Na legislatura anterior ja houve anos nos quais estas duas categorias chegaram a receber quase metade do total de assinaturas, sendo 46% do total em 2018 e 43% em 2019. E esta queda no percentual das mesmas no total de assinaturas continuou em 2020, indo para 23%, e posteriormente em 2021 passaram a ser 20,84%. Quando se trata de numeros absolutos, nos anos de 2017, 2018 e 2019, a area de datas comemorativas e homenagens diversas, em 2019 foi a mais assinada. Ja em 2020, quando desenvolvimento social foi a categoria mais assinada, ela caiu para a quarta colocacao e em 2021 foi para quinto lugar, quando a area de frente parlamentar foi a mais assinada. Por fim, outro ponto a destacar sao as assinaturas em projetos nas categorias de combate a corrupcao e transparencia. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br No tocante aos projetos de lei na area de combate a corrupcao a quantidade de assinaturas permaneceu baixa em 2021, com um total de 8 assinaturas, e os projetos nesta area giraram em torno da obrigatoriedade da prestacao de informacoes ao legislativo sobre alteracoes das tarifas do servico transporte publico de passageiros do municipio, da retomada da gestao administrativa e pedagogica de equipamentos repassados a gestao indireta por associacoes e OSCIPs para o provimento da educacao infantil de 0 a 3 anos de idade e da politica municipal de fiscalizacao, prevencao e combate a adulteracao e comercializacao de combustiveis adulterados. Quando comparada com 2017, ano em que esse valor de assinaturas era de 26, considera-se um numero baixo. E sobre os baixos numeros de assinaturas encontrados na area de combate a corrupcao, algumas suposicoes podem ser feitas baseadas tanto no contexto de epoca, assim como no desenrolar dos anos seguintes. Vale destacar que em meados de 2017 o Brasil ainda vivia as atividades da Operacao Lava Jato, que se apresentou como uma operacao que visava combater a corrupcao, sendo que mesma tambem foi uma das principais motivadoras do surgimento de individuos, grupos politicos e movimentos que defendiam o enfrentamento a praticas corruptas na sociedade. No entanto, o que se observou nos anos seguintes foi o progressivo esvaziamento desta causa, o que leva a supor que a bandeira do combate a corrupcao nao passou de mera retorica que serviu para promover os projetos de poder de alguns setores da sociedade. E isto nao foi diferente na Camara dos Vereadores de Sao Paulo, porque no ano de 2018 nao houve nenhuma assinatura em projetos para esta area, em 2019 este numero sobe para 1, em 2020 aumenta para 9 e em 2021 cai para 8. O dever de agora e observar se o numero de assinaturas em projetos de combate a corrupcao vai aumentar ou se vai estagnar ou diminuir. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br Ja o numero de assinaturas em projetos de lei na area de transparencia no ano de 2021 foi de 23 ao todo. Quando comparado com a sequencia historica, que surgiu em 2017, este chega a ser um numero alto para a area, ficando atras apenas do ano de 2019 quando houve 35 assinaturas ao todo. Na area de transparencia os projetos giraram em torno da consolidacao da politica de dados abertos e transparencia no ambito do municipio, combate a noticias falsas, digitalizacao de documentos fisicos do municipio, a criacao de um observatorio de dados abertos do municipio e divulgacao de informacoes relacionadas a obras, contaminacao por covid-19, vacinacao, contratos de terceirizacao de servicos e os processos administrativos relacionados a regularizacao fundiaria. Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br CONSIDERACOES FINAIS E valido e importante destacar que o exercicio do controle social e extremamente necessario para saber se as instituicoes estao em bom funcionamento e se mantem sintonia com as demandas e a realidade da sociedade, alem de ajudar a sociedade a propor caminhos para melhorar a gestao publica, a fomentar a participacao social na administracao publica e a levar conhecimento ao publico. A principio, nota-se que a Camara dos Vereadores no ano de 2021 esteve mais ativa na questao de proposicao de projetos legislativos que nos anos anteriores. Alem disso, pelo segundo ano os projetos propostos em areas de grande impacto permaneceram como as mais assinadas, a saber: desenvolvimento social e saude e esporte. Isto e muito positivo. No entanto e importante dar atencao para saber se isto e apenas uma questao mais relacionada a pandemia ou se realmente o poder legislativo municipal esta comecando a dar atencao a areas de grande importancia social, pois as areas de denominacao de logradouro e datas comemorativas e homenagens diversas ainda permanecem como algumas das categorias mais assinadas na Camara. Ja o ponto negativo esta a baixa assinatura em projetos de transparencia e de combate a corrupcao. E de grande importancia que os legisladores deem mais atencao a estas areas que possuem uma dependencia mutua, pois a baixa transparencia torna mais dificil a elaboracao e a avaliacao de politicas publicas, dificulta o combate a corrupcao e inviabiliza o controle social do poder publico pela sociedade.",
        "qas": [
            {
                "question": r"O que o relatório sugere sobre a relação entre transparência e controle social?",
                "id": "27",
                'is_impossible': False,
                "answers": [
                    {
                        "text": " Afirma que a baixa transparência dificulta o controle social e a avaliação das políticas públicas.",
                        "answer_start": 9
                    }
                ]
            }
        ]
    },
        {
        "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%. Em seguida aparecem os projetos na area de frente parlamentar foram as mais assinadas, com 271 assinaturas, representando 12,52% do total.",
        "qas": [
            {
                "question": r"Quais foram os principais temas abordados nos projetos de combate à corrupção em 2021?",
                "id": "28",
                'is_impossible': False,
                "answers": [
                    {
                        "text": "Alterações tarifárias no transporte público, gestão administrativa e fiscalização de combustíveis adulterados.",
                        "answer_start": 9
                    }
                ]
            }
        ]
    },
{
        "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%. Em seguida aparecem os projetos na area de frente parlamentar foram as mais assinadas, com 271 assinaturas, representando 12,52% do total.",
        "qas": [
            {
                "question": r"Como a vacinação influenciou as atividades legislativas em 2021?",
                "id": "29",
                'is_impossible': False,
                "answers": [
                    {
                        "text": "Permitiu o retorno gradativo das atividades presenciais nas instituições do Estado, incluindo a Câmara dos Vereadores.",
                        "answer_start": 9
                    }
                ]
            }
        ]
    },
        {
        "context": "Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%. Em seguida aparecem os projetos na area de frente parlamentar foram as mais assinadas, com 271 assinaturas, representando 12,52% do total.",
        "qas": [
            {
                "question": r"Quais reflexões o relatório traz sobre os projetos na área de desenvolvimento social em 2021?",
                "id": "30",
                'is_impossible': False,
                "answers": [
                    {
                        "text": " Ressalta que o aumento das assinaturas pode ser um indicativo de maior atenção a áreas de impacto social, embora ainda precise ser monitorado para verificar se é uma tendência ou apenas reflexo da pandemia.",
                        "answer_start": 9
                    }
                ]

            }
        ]
    }
]

In [ ]:
from  simpletransformers.question_answering import QuestionAnsweringModel

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install googletrans==4.0.0-rc1

In [ ]:
from transformers import BigBirdForQuestionAnswering, BigBirdTokenizer, pipeline

# Carregar o modelo e o tokenizer BigBird
model = BigBirdForQuestionAnswering.from_pretrained("google/bigbird-roberta-base")
tokenizer = BigBirdTokenizer.from_pretrained("google/bigbird-roberta-base")

# Usar o pipeline de question answering
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Treine o vectorizer com todos os documentos
vectorizer = TfidfVectorizer()
document_tfidf = vectorizer.fit_transform(chunks)  # 'chunks' é a lista de documentos

# Agora, transforme a consulta usando o mesmo vectorizer
def search_index(question, vectorizer, document_tfidf, document_titles, chunks):
    # Transforme a consulta com o mesmo vectorizer que foi ajustado para os documentos
    question_vector = vectorizer.transform([question])  # Transformação TF-IDF para a pergunta

    # Verifique as dimensões para garantir que estão alinhadas
    print("Dimensões do vetor da pergunta:", question_vector.shape)
    print("Dimensões dos documentos:", document_tfidf.shape)

    # Calcular a similaridade de cosseno entre o vetor da pergunta e os documentos
    from sklearn.metrics.pairwise import cosine_similarity

    # Calcular similaridade entre a consulta e os documentos
    similarities = cosine_similarity(question_vector, document_tfidf)

    # Ordenar os documentos com base na similaridade
    ranked_docs = similarities[0].argsort()[::-1]  # Classificar por similaridade decrescente

    # Retornar as frases mais relevantes
    lista_frases = {document_titles[i]: chunks[i] for i in ranked_docs[:5]}  # Pegando os top 5 documentos mais relevantes
    return lista_frases

def translate_text(text, src_lang='auto', dest_lang='en'):
    from googletrans import Translator

    # Inicializando o tradutor
    translator = Translator(service_urls=[
        'translate.google.com',
        'translate.google.co.kr',
    ])

    # Verificar se o texto é válido antes de tentar traduzir
    if not text:
        raise ValueError("O texto fornecido para tradução não pode ser vazio ou None.")

    # Traduzindo o texto
    result = translator.translate(text, src=src_lang, dest=dest_lang)

    # Retornando o texto traduzido
    return result.text


while True:
    question = input("Por favor, insira sua pergunta (ou 'sair' para terminar): ")
    if question.lower() == 'sair':
        break

    # Corrigir texto da pergunta
    question = corrigir_texto(question)

    # Buscar frases relevantes
    lista_frases = search_index(question, vectorizer, document_tfidf, document_titles, chunks)
    if not lista_frases:
        print("Nenhuma frase relevante encontrada.")
        continue

    # Selecionar o melhor trecho
    lista_frases = list(lista_frases.values())
    melhor_frase = select_best_chunk(lista_frases, question, vectorizer)
    print(melhor_frase)
    if melhor_frase:
      melhor_frase_en = translate_text(melhor_frase, src_lang='pt', dest_lang='en')
      print(melhor_frase_en)
    else:
      print("Não foi possível selecionar uma frase válida para tradução.")
    question_en = translate_text(question, src_lang='pt', dest_lang='en')

    # Obter a resposta
    result = qa_pipeline(question=question_en, context=melhor_frase_en)

    # Tradução da resposta de volta para o português
    result_pt = translate_text(result['answer'], src_lang='en', dest_lang='pt')

    print(f"Resposta: {result_pt}")


Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['qa_classifier.intermediate.dense.bias', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.output.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Por favor, insira sua pergunta (ou 'sair' para terminar): Quais categorias tiveram aumentos significativos nas assinaturas em 2021, quando comparadas a 2020?
Dimensões do vetor da pergunta: (1, 812)
Dimensões dos documentos: (15, 812)
Possivelmente os projetos propostos foram impactados pela pandemia de Covid-19, refletindo-se nas categorias de projetos que os vereadores mais assinaram ao longo do ano de 2021. Pela primeira vez, em numeros absolutos, aparecem os projetos na area de desenvolvimento social como os que tiveram um aumento significativo na quantidade de assinaturas, indo de 211 em 2020 para 271 em 2021. Em aspectos proporcionais, o valor tambem aumenta: as assinaturas em projetos propostos nas area de desenvolvimento social representava 10,14% do total em 2020, mas em 2021 representaram 12,66%.
Possibly the proposed projects were impacted by the COVID-19 pandemic, reflected in the project categories that the councilors signed the most throughout 2021. For the first time, in

Attention type 'block_sparse' is not possible if sequence_length: 384 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Resposta: Pela primeira vez,
Por favor, insira sua pergunta (ou 'sair' para terminar): Quais categorias tiveram aumentos significativos nas assinaturas em 2021, quando comparadas a 2020?
Dimensões do vetor da pergunta: (1, 524)
Dimensões dos documentos: (15, 812)


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 524 while Y.shape[1] == 812